<a href="https://colab.research.google.com/github/georgialoukatou/fyssen-project/blob/master/HAI_ling_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install git+https://github.com/georgialoukatou/Cornell-Conversational-Analysis-Toolkit.git/@Hai

import convokit
from convokit import Corpus, download

corpus = Corpus(download('subreddit-Cornell'))
!python -m spacy download en_core_web_sm


  Cloning https://github.com/georgialoukatou/Cornell-Conversational-Analysis-Toolkit.git/ (to revision Hai) to /tmp/pip-req-build-9mkp3gl1
  Running command git clone -q https://github.com/georgialoukatou/Cornell-Conversational-Analysis-Toolkit.git/ /tmp/pip-req-build-9mkp3gl1
  Running command git checkout -b Hai --track origin/Hai
  Switched to a new branch 'Hai'
  Branch 'Hai' set up to track remote branch 'Hai' from 'origin'.
     |████████████████████████████████| 12.7MB 334kB/s 
     |████████████████████████████████| 1.4MB 42.9MB/s 
     |████████████████████████████████| 245kB 32.3MB/s 
     |████████████████████████████████| 1.1MB 34.2MB/s 
     |████████████████████████████████| 9.1MB 50.2MB/s 
     |████████████████████████████████| 460kB 40.3MB/s 
     |████████████████████████████████| 133kB 50.8MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 
     |████████████████████████████████| 122kB 50.1MB/s 
  Created wheel for convokit: filename=convokit-2.4.4-cp37-none-

In [5]:
from convokit.text_processing import TextParser
textparser = TextParser()
textparser.transform(corpus)

In [118]:
#check use of PoS
import collections
convo = corpus.random_conversation()
d = collections.defaultdict(list)

d2 = collections.defaultdict(list)
for spkr in convo.iter_speakers():
   spkr_utts = list(spkr.iter_utterances())
   for i in spkr_utts:
      d[spkr.id].append(len(i.text.split(" "))) 
      for j in i.meta["parsed"]:
        d2[spkr.id].append(list())
        for k in range(len(j["toks"])) :
          d2[spkr.id][-1].append(j["toks"][k]["pos"]) 


for i in range(len(d2[spkr.id])):
    d2[spkr.id][i] = collections.Counter(d2[spkr.id][i])


#d2 gives parts of speech for each utterance and each speaker)
print(d2)

defaultdict(<class 'list'>, {'ssj4rit': [['INTJ'], ['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'PROPN', 'CCONJ', 'PRON', 'AUX', 'PART', 'VERB', 'NOUN', 'PART', 'VERB', 'NOUN', 'PUNCT'], ['PRON', 'VERB', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'PUNCT'], ['PRON', 'VERB', 'VERB', 'NOUN', 'ADP', 'PROPN'], ['CCONJ', 'PRON', 'ADV', 'AUX', 'PART', 'VERB', 'DET', 'NOUN', 'PUNCT', 'PRON', 'VERB', 'PUNCT', 'DET', 'NOUN', 'VERB', 'PUNCT', 'PUNCT'], ['PRON', 'VERB', 'PART', 'ADJ', 'ADV', 'ADV', 'PRON', 'AUX', 'VERB', 'ADP', 'NOUN', 'NOUN', 'PUNCT', 'PRON', 'VERB', 'PROPN', 'CCONJ', 'PRON', 'DET', 'VERB', 'VERB', 'PROPN', 'PUNCT'], ['ADV', 'PRON', 'ADV', 'VERB', 'ADP', 'PRON', 'NOUN', 'CCONJ', 'ADP', 'NOUN', 'NOUN', 'PRON', 'VERB', 'PRON', 'AUX', 'VERB', 'PART', 'VERB', 'VERB', 'NOUN', 'PUNCT'], ['AUX', 'DET', 'NOUN'], ['CCONJ', 'INTJ', 'PUNCT'], ['PRON', 'ADV', 'AUX', 'VERB', 'ADP', 'NOUN', 'ADV', 'CCONJ', 'VERB', 'ADP', 'VERB', 'PRON', 'NOUN', 'ADP', 'DET', 'NOUN'], ['CCONJ', 'PRON', 'ADV', 'ADV', 'VERB'

In [126]:
#check length of utterances
import pandas as pd

df=pd.DataFrame.from_dict(d,orient='index').T #length of utterance for each speaker

print(df)

     ssj4rit  blahdumb  Pihris  mushiettake
0        1.0      10.0    40.0          1.0
1       20.0      40.0    27.0        198.0
2       38.0      62.0     1.0          1.0
3       52.0      62.0     1.0          1.0
4       21.0       NaN    21.0          1.0
..       ...       ...     ...          ...
351      NaN       NaN     NaN        105.0
352      NaN       NaN     NaN        105.0
353      NaN       NaN     NaN         36.0
354      NaN       NaN     NaN         53.0
355      NaN       NaN     NaN         98.0

[356 rows x 4 columns]


In [185]:
# code minimally adapted from Doyle et al, 2016 / Yurovski et al., 2016

import operator, itertools
def group(utterances):
	utterances.sort(key=operator.itemgetter('convId'))
	list1 = []
	for key, items in itertools.groupby(utterances, operator.itemgetter('convId')):
		list1.append(list(items))
  
	return list1


def allMarkers(markers):
	categories = []
	for marker in markers:
		categories.append(marker)
	return categories

def checkMarkers(markers):
	toReturn = []
	for marker in markers:
		if isinstance(marker, str):
			toReturn.append({"marker": marker, "category": marker})
		else:
			toReturn.append(marker)
	return toReturn


def findMarkersInConvo(markers,convo):
	ba = {} # Number of times Person A and person B says the marker["marker"]
	bna = {}
	nbna = {}
	nba = {}
	for utterance in convo:				
		for j, marker in enumerate(markers):
			word = marker["marker"]
			msgMarker = word in utterance["msgMarkers"]
			replyMarker = word in utterance["replyMarkers"]
			
			if msgMarker and replyMarker:
				ba[word] = ba.get(word,0) + 1
			elif replyMarker and not msgMarker:
				bna[word] = bna.get(word,0) + 1
			elif not replyMarker and msgMarker:
				nba[word] = nba.get(word,0) + 1
			else:
				nbna[word] = nbna.get(word,0) + 1

			print("ggg", ba, bna, nba, nbna)
	return({'ba': ba,'bna': bna,'nba': nba,'nbna': nbna})
 
def metaDataExtractor(groupedUtterances, markers,corpusType=''):
	results = []
	for i, convo in enumerate(groupedUtterances):
				
		toAppend = findMarkersInConvo(markers,convo)		
		results.append(toAppend)
	return results

def readMarkers(markersFile,dialect=None):
	if dialect is None:
		reader = csv.reader(open(markersFile))
	else:
		reader = csv.reader(open(markersFile),dialect=dialect)
	markers = []
	#print('marker\tcategory')
	for i, row in enumerate(reader):
		toAppend = {}
		toAppend["marker"] = row[0]
		if(len(row) > 1):
			toAppend["category"] = row[1]
		else:
			toAppend["category"] = row[0]
		markers.append(toAppend)
		#print(toAppend["marker"]+'\t'+toAppend["category"])
	return markers

def writeFile(results, outputFile, shouldWriteHeader):
	if len(results) == 0:
		print("x")
		return
	toWrite = []
	header = sorted(list(results[0].keys()))
	for row in results:
		toAppend = []
		for key in header:
			toAppend.append(row[key])
		toWrite.append(toAppend)
	if shouldWriteHeader:
		with open(outputFile, "w", newline='') as f:
			writer = csv.writer(f)
			writer.writerows([header])
		f.close()
	with open(outputFile, "a", newline='') as f:
		writer = csv.writer(f)
		writer.writerows(toWrite)
	f.close()

def determineCategories(msgMarkers,catdict,useREs=False):
	msgCats = []
	#iterate over catdict items {category: [words/REs]}
	for cd in catdict.items():
		if useREs:
			if any(any(wordre.match(marker) for marker in msgMarkers) for wordre in cd[1]):	#if REs, see if any tokens match each RE
				msgCats.append(cd[0])
		else:
			if any(word in msgMarkers for word in cd[1]):			#if just words, see if any word in category also in msg
				msgCats.append(cd[0])
	return msgCats

def makeCatDict(markers,useREs=False):
	mdict = {}
	for m in markers:
		marker = re.compile(''.join([m["marker"], '$'])) if useREs else m["marker"]
		if m["category"] in mdict:
			mdict[m["category"]].append(marker)
		else:
			mdict[m["category"]] = [marker]
		#mdict[m["category"]] = mdict.get(m["category"],[]).append(m["marker"])	#Need to swap marker and category labels
		#mdict[m["marker"]] = mdict.get(m["marker"],[]).append(m["category"])
	return(mdict)
 
def createAlignmentDict(category,result,smoothing,corpusType=''):
	toAppend = {}
	print("R", result, category)
	ba = int(result["ba"].get(category, 0))
	bna = int(result["bna"].get(category, 0))
	nbna = int(result["nbna"].get(category, 0))
	nba = int(result["nba"].get(category, 0))
	print(ba,bna,nbna,nba)
	#Calculating alignment only makes sense if we've seen messages with and without the marker
	if (((ba+nba)==0 or (bna+nbna)==0)):
		return(None)
	
	toAppend["category"] = category
		
	#Calculating Echoes of Power alignment 
	powerNum = ba
	powerDenom = ba+nba
	baseNum = ba+bna
	baseDenom = ba+nba+bna+nbna

	if(powerDenom != 0 and baseDenom != 0):
		dnmalignment = powerNum/powerDenom - baseNum/baseDenom
		toAppend["dnmalignment"] = dnmalignment
	else:
		toAppend["dnmalignment"] = False
	
	powerNum = ba
	powerDenom = ba+nba
	baseDenom = bna+nbna
	baseNum = bna
	powerProb = math.log((powerNum+smoothing)/float(powerDenom+2*smoothing))
	baseProb = math.log((baseNum+smoothing)/float(baseDenom+2*smoothing))
	alignment = powerProb - baseProb
	toAppend["alignment"] = alignment
	
	toAppend["ba"] = ba
	toAppend["bna"] = bna
	toAppend["nba"] = nba
	toAppend["nbna"] = nbna
	print("toapp:", toAppend)
	return(toAppend)
 
def runFormula(results, markers, smoothing,corpusType):
	toReturn = []
	categories = allMarkers(markers)
	print("h", results) 
	print("c", categories)
	for i, result in enumerate(results):
		for j, category in enumerate(categories):
			toAppend = createAlignmentDict(category["category"],result,smoothing,corpusType)
			print("here", toAppend)   
			if toAppend is not None:
				toReturn.append(toAppend)
	toReturn = sorted(toReturn, key=lambda k: (k["speakerId"],k["replierId"],k["category"]))
	return toReturn

In [186]:
# pair utterances from convokit 
utterances =[]
markers = ["the", "of", "very", "to", "through", "and", "just", "a", "in"]

previous_utt = dict()
convo = corpus.random_conversation()

convo_utts= list(convo.iter_utterances())


for i in convo.traverse('bfs'):
  d = {}
  previous_utt[i.id] = (i.text,i.speaker.id)
  if i.reply_to == None:  
    continue
  d['convId'] = (i.speaker.id, i.reply_to)
  d['speakerId'] = previous_utt[i.reply_to][1]
  d['msg'] = previous_utt[i.reply_to][0]
  d['reply'] = i.text
  d['msgMarkers'] = []
  d['replyMarkers'] = []
  d['msgTokens']= []
  d['replyTokens']= []
  d["replierId"] = i.speaker.id
  utterances.append(d)

for idx, utt in enumerate(utterances):
  for marker in markers:
			if marker in utt['msg']:
				utterances[idx]['msgMarkers'].append(marker)
			if marker in utt['reply']:
				utterances[idx]['replyMarkers'].append(marker)


In [170]:
print(utterances)

[{'convId': ('dontich', 'wtdqb'), 'speakerId': 'Nacklefoodle', 'msg': 'As it is, the resources available to students interested in pursuing an interest or career in graphic design at Cornell are a terrible joke. A Graphic Design major does not exist in any school, nor is an interest cultivated or encouraged distally through approachable majors outside of CS or CIS. Merely Googling "graphic design Cornell" yields comically illustrative results:\n\n[The Program of Computer Graphics homepage,](http://www.graphics.cornell.edu/) terribly designed and last updated 9/2010.\n\n[The Design and Environmental Analysis homepage,](http://www.human.cornell.edu/dea/index.cfm) poorly designed with partially functioning Javascript (my favorite element is the huge blank square with "learn more!" in its corner).\n\n[If this is the best the Computing in the Arts faculty can come up with,](http://www.cis.cornell.edu/ComputingArts/) what is the point of pursuing graphic design at Cornell? Looks like someone

In [187]:
#main function - run alignment

smoothing=1
shouldWriteHeader=True
outputFile='aaaaaaaaa'
markers = checkMarkers(markers)
import math, csv
groupedUtterances = group(utterances)
print(groupedUtterances)
metaData = metaDataExtractor(groupedUtterances,markers) 
print(metaData)        
results = runFormula(metaData, markers, smoothing,'')
print(results)
writeFile(results, outputFile, shouldWriteHeader)


[[{'convId': ('Enyo287', '5hg9e7'), 'speakerId': 'PmMeYourFists', 'msg': 'The class grading just says 75% overall performance and 25% final exam, but theres no grades on anything throughout the term. We hand in homework and its corrected, but no score. Im freaking out. This class is insanely difficult. Help.', 'reply': "удачи, товарищ.\n\nI've taken a lot of language classes during my time at Cornell [20+ credits over 3 different languages] but in each there was extreme transparency. Given how the majority of the class grade is based on your overall performance [as most language classes are], you probably should have thought about this a little earlier, to ask the instructor.\n\nHowever, I can say that if you showed up to class almost everyday, did your best, did your homework, took your exams, etc., you'll probably be OK and will walk about with a B or better.\n\nSource: Someone who showed up to class almost everyday, did a little less than my best, did most of my homework, took my ex